# Use logical constraints with decision optimization

This tutorial includes everything you need to set up decision optimization engines, build a mathematical programming model, leveraging logical constraints.


When you finish this tutorial, you'll have a foundational knowledge of _Prescriptive Analytics_.

>This notebook is part of the **[Prescriptive Analytics for Python](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)**

>It requires a valid subscription to **Decision Optimization on the Cloud** or a **local installation of CPLEX Optimizers**. 

Discover us [here](https://developer.ibm.com/docloud)


Table of contents:

-  [Describe the business problem](#Describe-the-business-problem:--Games-Scheduling-in-the-National-Football-League)
*  [How decision optimization (prescriptive analytics) can help](#How--decision-optimization-can-help)
*  [Use decision optimization](#Use-decision-optimization)
    *  [Step 1: Download the library](#Step-1:-Download-the-library)
    *  [Step 2: Set up the engines](#Step-2:-Set-up-the-prescriptive-engine)
    *  [Step 3: Learn about constraint truth values](#Step-3:-Learn-about-constraint-truth-values)
    *  [Step 4: Learn about equivalence constraints](#Step-4:-Learn-about-equivalence-constraints)
*  [Summary](#Summary)
****

Logical constraints let you use the _truth value_ of constraints inside the model. The truth value of a constraint 
is true when it is satisfied and false when not. Adding a constraint to a model ensures that it is always satisfied. 
However, with logical constraints, one can use the truth value of a constraint _inside_ the model, allowing to choose dynamically whether a constraint is to be satisfied (or not).

## How  decision optimization can help

* Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes.  It takes into account specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

<u>With prescriptive analytics, you can:</u> 

* Automate the complex decisions and trade-offs to better manage your limited resources.
* Take advantage of a future opportunity or mitigate a future risk.
* Proactively update recommendations based on changing events.
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.



## Use decision optimization

### Step 1: Download the library

Run the following code to install Decision Optimization CPLEX Modeling library.  The *DOcplex* library contains the two modeling packages, Mathematical Programming and Constraint Programming, referred to earlier.

In [ ]:
import docplex
assert  (docplex.__version__ >= '2.4'), "Please update docplex"

A restart of the kernel might be needed.

### Step 2: Set up the prescriptive engine

* Subscribe to the [Decision Optimization on Cloud solve service](https://developer.ibm.com/docloud).
* Get the service URL and your personal API key.

In [ ]:
SVC_URL = "ENTER YOUR URL HERE"
SVC_KEY = "ENTER YOUR KEY HERE"

from docplex.mp.model import Model

### Step 3: Learn about constraint truth values

Any discrete linear constraint can be associated to a binary variable that holds the truth value of the constraint. 
But first, let's explain what a discrete constraint is

#### Discrete linear constraint

A discrete linear constraint is built from discrete coefficients and discrete variables, taht is variables with type `integer` or `binary`. For example, assuming x and y are integer variables:

 - `2x+3y == 1` is discrete
 - `x+y = 3.14` is not (because of 3.14)
 - `1.1 x + 2.2 y <= 3` is not because of the non-integer coefficients 1.1 and 2.2

#### The truth value of an added constraint is always 1

The truth value of a constraint is accessed by the `status_var` property. This varianle is aplain Docplex decision variable that can be used anywhere a variable can. However, the value of the truth value variable and the constraint are linked, both ways:

 - a constraint is satisfied if and only if its truth value variable equals 1
 - a constraint is _not_ satisfied if and only if its truth value variable equals 0.

In this toy model,we show that the truth value of a constraint which has been added to a model is always equal to 1.

In [ ]:
m1 = Model()
x = m1.integer_var(name='ix')
y = m1.integer_var(name='iy')
ct = m1.add(x + y <= 3)
ct_truth = ct.status_var
m1.maximize(x+y)
assert m1.solve()
print('the truth value of [{0!s}] is {1}'.format(ct, ct_truth.solution_value))

#### The truth value of a constraint not added to a model is undefined

A constraint that is not added to a model, has no effect. Its truth value is undefined: it can be either 1 or 0.

In the following example, both `x` and `y` are set to their upper bound, so that the constraint is not satisfied; hence the truth value is 0.

In [ ]:
m2 = Model()
x = m2.integer_var(name='ix', ub=4)
y = m2.integer_var(name='iy', ub=4)
ct = (x + y <= 3)
ct_truth = ct.status_var  # not m2.add() here!
m2.maximize(x+y)
assert m2.solve()
m2.print_solution()
print('the truth value of [{0!s}] is {1}'.format(ct, ct_truth.solution_value))

#### Using constraint truth values in modeling

A constraint's truth value is actually a plain DOcplex decision variable, and as such, can be used with comparison operators and arithmetic operators.
Let's experiment again with a toy model: in this model,
we state that the truth value of `y == 4` is less than the truth value of `x ==3`.
As we maximize y, y has value 4 in the optimal solution (it is the upper bound), and consequently the constraint `ct_y4` is satisfied. From the inequality between truth values,
it follows that the truth value of `ct_x2` equals 1 and x is equal to 2.

Using the constraints in the inequality has silently converted each constraint into its truth value.

In [ ]:
m3 = Model()
x = m3.integer_var(name='ix', ub=4)
y = m3.integer_var(name='iy', ub=4)
ct_x2 = (x == 2)
ct_y4 = (y == 4)
m3.add( ct_y4 <= ct_x2 )
m3.maximize(y)
assert m3.solve()
m3.print_solution()

Constraint truth values can be used with arithmetic operators, just as variables can. In th enext model, we express a more complex constraint:
- either x is equal to 3, _or_ both y and z are equal to 5

Let's see how we can express this easilty with truth values:

In [ ]:
m31 = Model(name='m31')
x = m31.integer_var(name='ix', ub=4)
y = m31.integer_var(name='iy', ub=10)
z = m31.integer_var(name='iz', ub=10)
ct_x2 = (x == 3)
ct_y5 = (y == 5)
ct_z5 = (z == 5)
#either ct-x2 is true or -both- ct_y5 and ct_z5 mus be true
m31.add( 2 * ct_x2 + (ct_y5 + ct_z5) == 2)
# force x to be less than 2: it cannot be equal to 3!
m31.add(x <= 2)
# maximize sum of x,y,z
m31.maximize(x+y+z)
assert m31.solve()
# the expected solution is: x=2, y=5, z=5
assert m31.objective_value == 12
m31.print_solution()

As we have seen, constraints can be used in expressions. This includes the `Model.sum()` and `Model.dot()` aggregation methods.

In the next model, we define ten variables, one of which must be equal to 3 (we dpn't care which one, for now). As we maximize the sum of all `xs` variables, all will end up equal to their upper bound, except for one.

In [ ]:
m4 = Model()
xs = m4.integer_var_list(10, ub=100)
cts = [xi==3 for xi in xs]
m4.add( m4.sum(cts) == 1)
m4.maximize(m4.sum(xs))
assert m4.solve()
m4.print_solution()

As we can see, all variables but one are set to their upper bound of 100. We cannot predict which variable will be set to 3. 
However, let's imagine that we prefer variable with a lower index to be set to 3, how can we express this preference? 

The answer is to use an additional expression to the objective, using a scalar product of constraint truth value

In [ ]:
preference = m4.dot(cts, (k+1 for k in range(len(xs))))
# we prefer lower indices for satisfying the x==3 constraint
# so the final objective is a maximize of sum of xs -minus- the preference
m4.maximize(m4.sum(xs) - preference)
assert m4.solve()
m4.print_solution()

As expected, the `x` variable set to 3 now is the first one.

#### Using truth values to state 'not equals' constraints.

Truth values can be used to express elegantly 'not equal' constraints, by forcing the truth value of an equality constraint to 0.

In the next model, we illustrate how an equality constraint can be negated by forcing its truth value to zero. This negation forbids y to be equal to 4, as it would be without this negation.
Finally, the objective is 7 instead of 8.

In [ ]:
m5 = Model()
x = m5.integer_var(name='ix', ub=4)
y = m5.integer_var(name='iy', ub=4)
# this is the equality constraint we want to negate
ct_y4 = (y == 4)
# forcing truth value to zero means the constraint is not satisfied.
negation = m5.add( ct_y4 == 0)
# maximize x+y should yield both variables to 4, but y cannot be equal to 4
# as such we expect y to be equal to 3
m5.maximize(x + y)
assert m5.solve()
m5.print_solution()
# expecting 7 as objective, not 8
assert m5.objective_value == 7

# now remove the negation
m5.remove_constraint(negation)
# and solve again
assert m5.solve()
# the objective is 8 as expected: both x and y are equal to 4
assert m5.objective_value == 8

#### Summary

We have seen that linear constraints have an associated binary variable, its _truth value_, whose value is linked to whether or not the constraint is satisfied. 
Moreover, this llink enables to express 'not equals' constraints.# now remove netation

### Step 4: Learn about equivalence constraints

As we have seen, using a constraint in expressions automtically generates a truth value variable, whose value is linked to the status of the constraint. 

However, in some cases, it can be useful to relate the status of a constraint to an _existing_ binary variable. This is the purpose of equivalence constraints.

An equiavelnec constraints relates an existing binary variable to the status of a discrete linear constraints. The syntax is:

    `Model.add_equivalence(bvar, linear_ct, active_value, name)`
    
 - `bvar` is the existing binary variable
 - `linear-ct` is a discrete linear constraint
 - `active_value` can take values 1 or 0 (the default is 1)
 - `name` is an optional string to name the equivalence.

In [ ]:
m6 = Model(name='m6')
size = 7
il = m6.integer_var_list(size, name='i', ub=10)
jl = m6.integer_var_list(size, name='j', ub=10)
bl = m6.binary_var_list(size, name='b')
for k in range(size):
    # for each i, relate bl_k to il_k==5 *and* jl_k == 7
    m6.add_equivalence(bl[k], il[k] == 5)
    m6.add_equivalence(bl[k], jl[k] == 7)
# now maximize sum of bs
m6.maximize(m6.sum(bl))
assert m6.solve()
m6.print_solution()



## Summary


You learned how to set up and use the IBM Decision Optimization CPLEX Modeling for Python to formulate a Mathematical Programming model with logical constraints.

#### References
* [Decision Optimization CPLEX Modeling for Python documentation](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)
* [Decision Optimization on Cloud](https://developer.ibm.com/docloud/)
* Need help with DOcplex or to report a bug? Please go [here](https://developer.ibm.com/answers/smartspace/docloud)
* Contact us at dofeedback@wwpdl.vnet.ibm.com"


Copyright © 2017 IBM. Sample Materials.